In [1]:
from ast import literal_eval
import pandas as pd
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
res = pd.read_csv('data/match_results.csv')

In [3]:
for col in ['w1_or', 'w2_or', 'w3_or', 'w4_or', 
            'w1_nr', 'w2_nr', 'w3_nr', 'w4_nr', 
            'l1_or', 'l2_or', 'l3_or', 'l4_or', 
            'l1_nr', 'l2_nr', 'l3_nr', 'l4_nr', 
           ]:
    res[col].fillna(0, inplace=True)
    res[col] = res[col].astype(int)
    

In [4]:
start = pd.read_csv('data/match_starts.csv')

In [5]:
start = start.drop_duplicates(subset=['match_id'])

In [6]:
combined = res.merge(start, on='match_id', how='inner')

In [7]:
def w_players(row):
    if row['team_a_won']:
        players = literal_eval(row['team_a_players'])
    else:
        players = literal_eval(row['team_b_players'])
    players = [str(i) for i in players]
    players = players + [''] * (4 - len(players))
    players = '|'.join(players)
    return players


In [8]:
def l_players(row):
    if row['team_a_won']:
        players = literal_eval(row['team_b_players'])
    else:
        players = literal_eval(row['team_a_players'])
    players = [str(i) for i in players]
    players = players + [''] * (4 - len(players))
    players = '|'.join(players)
    return players


In [9]:
combined['winners'] = combined.apply(w_players, axis=1)
combined['losers'] = combined.apply(l_players, axis=1)

In [10]:
combined.head()

,match_id,w1,w2,w3,w4,l1,l2,l3,l4,w1_or,...,l3_nr,l4_nr,team_a_won,team_b_won,date,map,team_a_players,team_b_players,winners,losers
0,842505,addu,Livi,Shadeslayer II,Avenger,OS,Av3stan,avnish1,NewHaveScouts,2530,...,888,827,True,False,2024-01-22,Ghost Lake,"[657452537506562069, 695309066771365970, 50925...","[495896198444613643, 296465677114671105, 46186...",657452537506562069|695309066771365970|50925134...,495896198444613643|296465677114671105|46186559...
1,842472,OS,Av3stan,Livi,Avenger,DOOMSTER,TalentButNoFarm,CHICKEN,pHANTOM,2060,...,846,584,True,False,2024-01-22,African Clearing (Random Civs),"[495896198444613643, 296465677114671105, 69530...","[314461722817593345, 348849437101260801, 69894...",495896198444613643|296465677114671105|69530906...,314461722817593345|348849437101260801|69894350...
2,842458,addu,kappa9059,BraveHeart,pHANTOM,TalentButNoFarm,SaraSaiyan,MODI,Sephiroth,2511,...,1279,1273,False,True,2024-01-22,Oasis(Random Civs),"[348849437101260801, 699611550423711766, 69786...","[657452537506562069, 454538560348094466, 63503...",657452537506562069|454538560348094466|63503334...,348849437101260801|699611550423711766|69786207...
3,842448,DOOMSTER,Shadeslayer II,Avenger,pHANTOM,OS,Ambush,Livi,BraveHeart,3098,...,1179,880,True,False,2024-01-22,Land Nomad (Random Civs),"[314461722817593345, 509251341105889280, 22387...","[495896198444613643, 410863622139412482, 69530...",314461722817593345|509251341105889280|22387552...,495896198444613643|410863622139412482|69530906...
4,842442,OS,kappa9059,Ambush,Shadeslayer II,DOOMSTER,MODI,Livi,BraveHeart,2060,...,1194,895,True,False,2024-01-22,Kawasan (Random Civs),"[495896198444613643, 454538560348094466, 41086...","[314461722817593345, 697862073173934161, 69530...",495896198444613643|454538560348094466|41086362...,314461722817593345|697862073173934161|69530906...


In [11]:
from collections import defaultdict

In [12]:
player_map = defaultdict(list)

In [13]:
def map_players(row):
    winners = [int('0' + i) for i in row['winners'].split('|')]
    losers = [int('0' + i) for i in row['losers'].split('|')]
    w1, w2, w3, w4 = row['w1'], row['w2'], row['w3'], row['w4']
    l1, l2, l3, l4 = row['l1'], row['l2'], row['l3'], row['l4']
    for a, b in zip(winners, [w1, w2, w3, w4]):
        player_map[a].append(b)
    
    for a, b in zip(losers, [l1, l2, l3, l4]):
        player_map[a].append(b)


In [14]:
combined.apply(map_players, axis=1)

0       None
1       None
2       None
3       None
4       None
        ... 
6764    None
6765    None
6766    None
6767    None
6768    None
Length: 6769, dtype: object

In [15]:
player_map_final = {}
for k, v in player_map.items():
    player_map[k] = Counter(v).most_common()[0][0]

In [16]:
len(player_map)

166

In [17]:
def id_to_name(ids):
    x = []
    for i in ids.split('|'):
        try:
            x.append(player_map[int(i)])
        except:
            x.append('')
    return '|'.join(x)
combined['winners'] = combined['winners'].apply(id_to_name)
combined['losers'] = combined['losers'].apply(id_to_name)

In [18]:
combined.drop(['w1', 'w2', 'w3', 'w4', 'l1', 'l2', 'l3', 'l4', 
               'team_a_won', 'team_b_won', 
               'team_a_players', 'team_b_players'], 
              axis=1, inplace=True)

In [19]:
combined.head(43).tail()

,match_id,w1_or,w2_or,w3_or,w4_or,l1_or,l2_or,l3_or,l4_or,w1_nr,...,w3_nr,w4_nr,l1_nr,l2_nr,l3_nr,l4_nr,date,map,winners,losers
38,840110,1557,1521,1441,245,1793,1757,800,422,1577,...,1461,265,1773,1737,780,402,2024-01-19,Land Nomad (Random Civs),Thiru|Lathkore|MrWrong|ABHI,fenrir05|Valar|CHARITH|Nicco
39,840074,1537,1501,1421,225,1777,1649,820,442,1557,...,1441,245,1757,1629,800,422,2024-01-19,Arabia,Thiru|Lathkore|MrWrong|ABHI,Valar|Deadkorpse|CHARITH|Nicco
40,840046,1754,816,813,797,1672,1560,740,248,1777,...,836,820,1649,1537,717,225,2024-01-19,Valley,Valar|newisyou|Ju$TK!Dd!nG|CHARITH,Deadkorpse|Thiru|Rohan|ABHI
41,840012,1543,799,796,780,1771,1372,459,265,1560,...,813,797,1754,1355,442,248,2024-01-19,Nomad (Random Civs),Thiru|newisyou|Ju$TK!Dd!nG|CHARITH,Valar|KhalDrogo|Nicco|ABHI
42,839982,2987,1629,1511,1033,2669,1994,1284,1234,3008,...,1532,1054,2648,1973,1263,1213,2024-01-19,African Clearing (Random Civs),DOOMSTER|🥭bush|Royal|Avenger,addu|OS|Sephiroth|steelskin


In [20]:
c = combined.head(43)
c[['w1', 'w2', 'w3', 'w4']] = c['winners'].str.split('|', expand=True)


/var/folders/wk/xh1t9b793w52fjy9qr_0q3pr0000gn/T/ipykernel_31968/58338434.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c[['w1', 'w2', 'w3', 'w4']] = c['winners'].str.split('|', expand=True)
/var/folders/wk/xh1t9b793w52fjy9qr_0q3pr0000gn/T/ipykernel_31968/58338434.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c[['w1', 'w2', 'w3', 'w4']] = c['winners'].str.split('|', expand=True)
/var/folders/wk/xh1t9b793w52fjy9qr_0q3pr0000gn/T/ipykernel_31968/58338434.py:2: SettingWithCopyWarning: 
A value is tr

In [21]:
combined[['w1', 'w2', 'w3', 'w4']] = combined['winners'].str.split('|', expand=True)
combined[['l1', 'l2', 'l3', 'l4']] = combined['losers'].str.split('|', expand=True)
                                                                  

In [22]:
combined.drop(['winners', 'losers'], axis=1, inplace=True)

In [23]:
combined['year_month'] = combined['date'].apply(lambda x: x[:7])

In [24]:
combined = combined[['match_id', 'date', 'year_month', 'map', 
                     'w1', 'w2', 'w3', 'w4', 
                     'l1', 'l2', 'l3', 'l4', 
                     'w1_or', 'w2_or', 'w3_or', 'w4_or', 
                     'w1_nr', 'w2_nr', 'w3_nr', 'w4_nr', 
                     'l1_or', 'l2_or', 'l3_or', 'l4_or', 
                     'l1_nr', 'l2_nr', 'l3_nr', 'l4_nr'
                    ]]

In [25]:
combined

,match_id,date,year_month,map,w1,w2,w3,w4,l1,l2,...,w3_nr,w4_nr,l1_or,l2_or,l3_or,l4_or,l1_nr,l2_nr,l3_nr,l4_nr
0,842505,2024-01-22,2024-01,Ghost Lake,addu,Livi,Shadeslayer II,Avenger,OS,Av3stan,...,1171,1087,2091,2010,900,839,2079,1998,888,827
1,842472,2024-01-22,2024-01,African Clearing (Random Civs),OS,Av3stan,Livi,Avenger,DOOMSTER,talentbutnofarm,...,1210,1075,3113,1808,877,615,3082,1777,846,584
2,842458,2024-01-22,2024-01,Oasis(Random Civs),addu,Kappa,BraveHeart,pHANTOM,talentbutnofarm,Saracen,...,899,615,1827,1399,1298,1292,1808,1380,1279,1273
3,842448,2024-01-22,2024-01,Land Nomad (Random Civs),DOOMSTER,Shadeslayer II,Avenger,pHANTOM,OS,🥭bush,...,1044,596,2075,1614,1194,895,2060,1599,1179,880
4,842442,2024-01-22,2024-01,Kawasan (Random Civs),OS,Kappa,🥭bush,Shadeslayer II,DOOMSTER,MODI,...,1614,1144,3113,1313,1209,910,3098,1298,1194,895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6764,394835,2022-01-19,2022-01,African Clearing,AgentP97,BoseDK,wArDaDdY,Simba,Thiru,talentbutnofarm,...,1255,941,1787,1655,1194,877,1773,1641,1180,863
6765,394824,2022-01-19,2022-01,Golden Pit,Thiru,Mirror,Livi,BraveHeart,BoseDK,Himansu,...,1191,778,1400,1303,1252,870,1389,1286,1241,859
6766,394816,2022-01-19,2022-01,Arena,SMR,Thiru,BraveHeart,Bondulkar,talentbutnofarm,Himansu,...,767,767,1668,1322,1318,1265,1655,1303,1305,1252
6767,394792,2022-01-19,2022-01,African Clearing,Morty,Thiru,shrp,CrazyGuY,SMR,Afterlife,...,1268,1202,2335,1320,1185,756,2333,1318,1183,754


In [26]:
def fix_map_name(map_):
    random_civs = False
    if 'random civ' in map_.lower():
        random_civs = True
    map_name = map_.split('(')[0].split('-')[0].strip().title()
    if random_civs:
        map_name += ' (Random civs)'
    return map_name
    

In [27]:
def fix_map_only_name(map_):
    return map_.lower().split('(')[0].split('-')[0].strip().title()
    

In [28]:
combined['map'] = combined['map'].apply(fix_map_name)
combined['map_only'] = combined['map'].apply(fix_map_only_name)

In [29]:
combined.to_csv('data/matches.csv', index=False)

In [30]:
combined

,match_id,date,year_month,map,w1,w2,w3,w4,l1,l2,...,w4_nr,l1_or,l2_or,l3_or,l4_or,l1_nr,l2_nr,l3_nr,l4_nr,map_only
0,842505,2024-01-22,2024-01,Ghost Lake,addu,Livi,Shadeslayer II,Avenger,OS,Av3stan,...,1087,2091,2010,900,839,2079,1998,888,827,Ghost Lake
1,842472,2024-01-22,2024-01,African Clearing (Random civs),OS,Av3stan,Livi,Avenger,DOOMSTER,talentbutnofarm,...,1075,3113,1808,877,615,3082,1777,846,584,African Clearing
2,842458,2024-01-22,2024-01,Oasis (Random civs),addu,Kappa,BraveHeart,pHANTOM,talentbutnofarm,Saracen,...,615,1827,1399,1298,1292,1808,1380,1279,1273,Oasis
3,842448,2024-01-22,2024-01,Land Nomad (Random civs),DOOMSTER,Shadeslayer II,Avenger,pHANTOM,OS,🥭bush,...,596,2075,1614,1194,895,2060,1599,1179,880,Land Nomad
4,842442,2024-01-22,2024-01,Kawasan (Random civs),OS,Kappa,🥭bush,Shadeslayer II,DOOMSTER,MODI,...,1144,3113,1313,1209,910,3098,1298,1194,895,Kawasan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6764,394835,2022-01-19,2022-01,African Clearing,AgentP97,BoseDK,wArDaDdY,Simba,Thiru,talentbutnofarm,...,941,1787,1655,1194,877,1773,1641,1180,863,African Clearing
6765,394824,2022-01-19,2022-01,Golden Pit,Thiru,Mirror,Livi,BraveHeart,BoseDK,Himansu,...,778,1400,1303,1252,870,1389,1286,1241,859,Golden Pit
6766,394816,2022-01-19,2022-01,Arena,SMR,Thiru,BraveHeart,Bondulkar,talentbutnofarm,Himansu,...,767,1668,1322,1318,1265,1655,1303,1305,1252,Arena
6767,394792,2022-01-19,2022-01,African Clearing,Morty,Thiru,shrp,CrazyGuY,SMR,Afterlife,...,1202,2335,1320,1185,756,2333,1318,1183,754,African Clearing
